# Importation des packages

In [0]:
from google.colab import drive
import pickle
import dill as pickle
import pandas as pd
import spacy
import re
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
tf.enable_eager_execution()

In [0]:
#!pip install -U spacy
#!python -m spacy download fr_core_news_sm
nlp = spacy.load('fr_core_news_sm')

## Lecture des fichiers contenant les données, les tokenisers, les modèles

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1Fdn0gB46wRs4sO0GJxBt5wg2ZaJSlsZo'
downloaded = drive.CreateFile({'id': '1Fdn0gB46wRs4sO0GJxBt5wg2ZaJSlsZo'})
downloaded.GetContentFile('matrice_td_faq')
matrice_td_faq = pickle.load(open('matrice_td_faq', 'rb'))

id = '1MmqR0VHIXGHS2qM4gILRQFM_IMmcrMTO'
downloaded = drive.CreateFile({'id': '1MmqR0VHIXGHS2qM4gILRQFM_IMmcrMTO'})
downloaded.GetContentFile('vectorizer_faq')
vectorizer_faq = pickle.load(open('vectorizer_faq', 'rb'))

id = '1kRF0HN8Cj91UKTDDuSjipzoEpQlUrMGj'
downloaded = drive.CreateFile({'id': '1kRF0HN8Cj91UKTDDuSjipzoEpQlUrMGj'})
downloaded.GetContentFile('classif_binaire_proba')
model_binaire = pickle.load(open('classif_binaire_proba', 'rb'))

id = '14TJXTz6P33D_xSrC9zXRONIuNk0prf5E'
downloaded = drive.CreateFile({'id': '14TJXTz6P33D_xSrC9zXRONIuNk0prf5E'})
downloaded.GetContentFile('bin_vectorizer_proba')
vector_binaire = pickle.load(open('bin_vectorizer_proba', 'rb'))

id = '1LpPR8X-0uD0_LfBEdfaz-2pooafNGUnX'
downloaded = drive.CreateFile({'id': '1LpPR8X-0uD0_LfBEdfaz-2pooafNGUnX'})
downloaded.GetContentFile('faq')
faq = pickle.load(open('faq', 'rb'))

id = '1iedCQnms2qJk9uMIQgbKIeJhzNzmLBte'
downloaded = drive.CreateFile({'id': '1iedCQnms2qJk9uMIQgbKIeJhzNzmLBte'})
downloaded.GetContentFile('texts')
texts = pickle.load(open('texts', 'rb'))

id = '1D4-dPKsNrwFOREKOyS_vY2sw94FMfnmG'
downloaded = drive.CreateFile({'id': '1D4-dPKsNrwFOREKOyS_vY2sw94FMfnmG'})
downloaded.GetContentFile('model_caractCOURT.h5')

id = '13jBzx8HcS3tv_OmybrbCcCQQfdW8mbGD'
downloaded = drive.CreateFile({'id': '13jBzx8HcS3tv_OmybrbCcCQQfdW8mbGD'})
downloaded.GetContentFile('idx2carCOURT')
idx2char = pickle.load(open('idx2carCOURT', 'rb'))

id = '16xSISZnJLZ4bEKyF4BJWYY_OpfeAxQil'
downloaded = drive.CreateFile({'id': '16xSISZnJLZ4bEKyF4BJWYY_OpfeAxQil'})
downloaded.GetContentFile('car2idxCOURT')
char2idx = pickle.load(open('car2idxCOURT', 'rb'))

## Importations pour la génération de texte

In [5]:
model = tf.keras.models.load_model('model_caractCOURT.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            58880     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 230)            235750    
Total params: 4,229,862
Trainable params: 4,229,862
Non-trainable params: 0
_________________________________________________________________


# Evaluation quantitative

Importation du jeu de données test.

In [0]:
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
id = '1wCRYVRVDJMgsHIhbpw0ndacMwaG0QMUM'
downloaded = drive.CreateFile({'id': '1wCRYVRVDJMgsHIhbpw0ndacMwaG0QMUM'})
downloaded.GetContentFile('Jeu_test')

jeu_test = pd.read_excel('Jeu_test')

In [0]:
jeu_test.columns = ['Questions', 'Reponses']

In [0]:
jeu_test['Questions'] = [phrase.strip('"').strip("'") for phrase in jeu_test['Questions']]
jeu_test['Reponses'] = [phrase.strip('"').strip("'") for phrase in jeu_test['Reponses']]

Fonction qui permet de lemmatiser le texte

In [0]:
def lem(text):
    """ Met à l'infinitif les verbes et met la racine des mots """
    tw_nlp = nlp(text) # spacy
    tokens = []
    for token in tw_nlp:
        tokens.append(token.lemma_)
    return ' '.join(tokens)

def substitute_url(text, url_replacement='URLEXPR'):
  txt = re.sub(r'(.+?)?(http.?://.+?)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",text,flags=re.IGNORECASE)
  for i in range(4) : 
    txt = re.sub(r'(.+?)?(http.?://.+?)\s?(.+?)?',r"\1"+url_replacement+' '+r"\3",txt,flags=re.IGNORECASE)
  return txt

def substitute_agence(text, replacement='AGENCE'):
  txt = re.sub(r'(.+?)?(tui.fr)\s?(.+?)?',r"\1"+replacement+' '+r"\3",text,flags=re.IGNORECASE)
  for i in range(4) : 
    txt = re.sub(r'(.+?)?(tui.fr)\s?(.+?)?',r"\1"+replacement+' '+r"\3",txt,flags=re.IGNORECASE)
    txt = re.sub(r'(.+?)?(tui)\s?(.+?)?',r"\1"+replacement+' '+r"\3",txt,flags=re.IGNORECASE)
    txt = re.sub(r'(.+?)?(exotismes)\s?(.+?)?',r"\1"+replacement+' '+r"\3",txt,flags=re.IGNORECASE)
  return txt 

In [0]:
def eval_quanti(ph_entree) : 
  """Prend une phrase en entrée et renvoie ce que le robot renvoie"""
  q = pd.DataFrame([ph_entree])
  q = q[0].apply(lem).apply(substitute_url).apply(substitute_agence)
  q_bin = vector_binaire.transform(q)
  q_sim = vectorizer_faq.transform(q)

  # si le modèle prédit 1 (c-a-d que la question de l'utilisateur rentre dans la thématique métier)
  if (model_binaire.predict_proba(q_bin)[0][0]<0.85):
    query_corpus_sim = np.squeeze(cosine_similarity(matrice_td_faq, q_sim))
    idx_most_sim = np.argmax(query_corpus_sim)
    ok = False
    for cl, questions in faq.items():
      for question in questions.items():
        # si la question de l'utilisateur similaire à une question 
        if str(question[0]) == texts[idx_most_sim] : # and 
          # alors on affiche la réponse à cette question
          return question[1]
    # sinon on affiche la réponse qui est la plus similaire
    if ok == False :
      return texts[idx_most_sim]
  # si le classifieur binaire prédit que la question de l'utilisateur n'est pas dans la thématique métier, alors on génère du texte 
  else : 
    return None

Calcul du score sur le jeu de test.

In [13]:
def scores(ph_entrees, ph_sorties) : 
  """ Calcul et renvoie l'accuracy suivant un jeu de données test """
  col_res = []
  for idx in range(len(ph_entrees)) : 
    print("#####################")
    print("Q :", ph_entrees[idx])
    print("RV :", ph_sorties[idx])
    print("RR : ", eval_quanti(ph_entrees[idx]))
    if eval_quanti(ph_entrees[idx]) ==  ph_sorties[idx] : # renvoie la bonne phrase
      col_res.append(1)
    else : # ne renvoie pas la bonne phrase
      col_res.append(0)
  
  accuracy = sum(col_res)/len(col_res)
  
  return accuracy

scores(jeu_test['Questions'], jeu_test['Reponses'])

#####################
Q : La location ne répondait absolument pas à mes attentes.
RV : Si vous souhaitez formuler une réclamation sur votre location de vacances, nous vous invitons à contacter directement notre partenaire TUI Villas au 01 74 18 03 76 tel:+33174180376 ou aller directement sur leur site sur la page "À propos de TUI Villas" rubrique "Question au service clientèle".
RR :  None
#####################
Q : Je ne suis pas contente de mes vacances
RV : Si vous souhaitez formuler une réclamation sur votre location de vacances, nous vous invitons à contacter directement notre partenaire TUI Villas au 01 74 18 03 76 tel:+33174180376 ou aller directement sur leur site sur la page "À propos de TUI Villas" rubrique "Question au service clientèle".
RR :  None
#####################
Q : Mes vacances se sont mal passées, je veux me faire rembourser !
RV : Si vous souhaitez formuler une réclamation et obtenir une réponse, nous vous invitons à nous adresser un courrier (pas de traitement pa

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['agence', 'urlexpr'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


#####################
Q : Quels sont les circuits proposés en Espagne ?
RV : Rendez-vous sur la page circuits https://www.tui.fr/circuits/ du site TUI, puis dans la destination vous intéresse. Sur certains circuits, vous trouverez des informations pratiques et précises sur vos voyages : l'équipement à prévoir, des recommandations sur le pays, une bibliographie... Pour toutes informations complémentaires, nos conseillers vous accueillent au 0825 000 825 (Service 0,20€/min + prix appel Du lun. au sam. de 9h à 21h30, le dim. et jours fériés 10h/20h).
RR :  Cliquez ici https://www.tui.fr/devis-groupe/ et retrouvez toutes les informations concernant les demandes de groupe (à partir de 10 adultes).
#####################
Q : Où faut-il aller pour réserver le circuit que l'on souhaite faire ?
RV : Cliquez ici https://www.tui.fr/circuits/, consultez, réservez ! Vous pouvez également contacter un de nos conseillers voyages qui vous accueillera au 0825 000 825 (Service 0,20€/min + prix appel Du l

0.3230769230769231